# Load the needed data from Foundry.



In [ ]:
# install the foundry and pymatgen
!pip install foundry_ml
!pip install pymatgen

In [ ]:
# initiate the foundry
from foundry import Foundry
f = Foundry(no_browser=True, no_local_server=True)

Please paste the following URL in a browser:
https://auth.globus.org/v2/oauth2/authorize?client_id=984464e2-90ab-433d-8145-ac0215d26c8e&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=urn%3Aglobus%3Aauth%3Ascope%3Adata.materialsdatafacility.org%3Aall+https%3A%2F%2Fauth.globus.org%2Fscopes%2Fc17f27bb-f200-486a-b785-2a25e82af505%2Fconnect+urn%3Aglobus%3Aauth%3Ascope%3Asearch.api.globus.org%3Asearch+https%3A%2F%2Fauth.globus.org%2Fscopes%2F56ceac29-e98a-440a-a594-b41e7a084b62%2Fall+urn%3Aglobus%3Aauth%3Ascope%3Atransfer.api.globus.org%3Aall+https%3A%2F%2Fauth.globus.org%2Fscopes%2F81fc4156-a623-47f2-93ad-7184118226ba%2Fauth+openid+https%3A%2F%2Fauth.globus.org%2Fscopes%2Ffacd7ccc-c5f4-42aa-916b-a0e270e2c2a9%2Fall&state=_default&response_type=code&code_challenge=qRhicK9FKttaJ9A5U83hwtpvOy4wbIFEmrCYH2Yu0Ro&code_challenge_method=S256&access_type=offline&prefill_named_grant=Foundry+Login
Please Paste your Auth Code Below: 
3YSi12S3hzsW3dmT8ZggNiscuP6ORr
Please paste th

In [ ]:
f.list()

,source_id,name,year
0,_test_foundry_aflow_band_gaps_v1.1,AFlowLib Band Gap Data Package,2020
1,_test_blaiszik_jarvis_mbj_3d_v1.1,Jarvis MBJ 3D,2020
2,_test_foundry_fashion_mnist_v1.1,Fashion-MNIST: a Novel Image Dataset for Bench...,2021
3,_test_blaiszik_foundry_iris_v2.1,Foundry - Iris Dataset,2021
4,_test_foundry_stein_bandgaps_v1.1,JCAP images and absorption spectra for 179072 ...,2021
5,_test_foundry_oqmd_v1.1,Foundry - OQMD,2021
6,_test_foundry_experimental_bandgap_v1.1,Foundry - Experimental Bandgaps,2021
7,_test_foundry_aflow_bandgaps_v1.1,Foundry - Experimental Bandgaps,2021
8,_test_foundry_nrel_bandgap_v1.1,NREL MatDB Band Gaps,2021
9,_test_foundry_oqmd_bandgap_v1.1,Foundry - OQMD Bandgaps,2021


Load and clean the experimental data. For clean method details, please refer to the paper

In [ ]:
import numpy as np
from pymatgen.core import Structure
import pandas as pd

f = f.load('_test_exp_bandgap_v1.1', globus=False)
df_X, df_Y = f.load_data()
N = len(df_Y)
drop_index = []
for i in range(N):
    if type(df_Y.loc[i,'bandgap value (eV)']) == list:
        new = [float(k) for k in df_Y.loc[i,'bandgap value (eV)']]
        mean = np.mean(new)
        std = np.std(new)
        if abs(mean) > 1.e-6 and std/mean > 0.1:
            drop_index.append(i)
        else:
            df_Y.loc[i,'bandgap value (eV)'] = mean
df1_exp_Y = df_Y.drop(drop_index)
df1_exp_X = df_X.drop(drop_index)
# select 300K, optical, smallest bandgap data
composition = []
sp = []
N1 = len(df1_exp_Y)
for i in range(N1):
    str_temp = Structure.from_dict(df1_exp_X.iloc[i]['structure'])
    composition.append(str_temp.composition.reduced_formula)
    sp.append(str_temp.get_space_group_info()[1])
#     composition.append(i['Composition'])
df_300_smallest_X = []
df_300_smallest_Y = []
for com in list({value:"" for value in composition}):
    temp = []
    for index in range(N1):
        if composition[index] == com:
            temp.append(index)
    spacegroup = []
    for index in temp:
        spacegroup.append(sp[index])
#     print(spacegroup, list(set(spacegroup)))
    for sp1 in list({value:"" for value in spacegroup}):
        temp1 = []
        kinds = []
        for index in temp:
            if  sp[index] == sp1:
                temp1.append(index)
                string = df1_exp_X.iloc[index]['temp (K)'] + df1_exp_X.iloc[index]['bandgap type'] + df1_exp_X.iloc[index]['exp method']
                kinds.append(string)
#         print('temp1',temp1)
        for case in ['300IO','300I','IO','I','300O','300','O','','300DO','300D','DO','D']:
            if case in kinds:
                for index in temp1:
                    string = df1_exp_X.iloc[index]['temp (K)'] + df1_exp_X.iloc[index]['bandgap type'] + df1_exp_X.iloc[index]['exp method']
                    if string == case:
                        df_300_smallest_X.append(df1_exp_X.iloc[index])
                        df_300_smallest_Y.append(df1_exp_Y.iloc[index])
                break
            else:
                continue      
print(len(df_300_smallest_X))
df_exp_X = pd.DataFrame(df_300_smallest_X)
df_exp_Y = pd.DataFrame(df_300_smallest_Y)
df_exp_X, df_exp_Y

Loading by source_id
Done curling.
[{'/foundry/_test_exp_bandgap_v1.1/foundry_dataframe.json status': True}]


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1464


(     reference        icsd_id  ... space group temp (K)
 0           41  MPDS-S1531436  ...       Fd-3m         
 1            1         238499  ...        Pnma      300
 2            1         416461  ...       Pm-3m         
 3            1         262893  ...      Cmc2_1         
 4            1          50460  ...        Fddd         
 ...        ...            ...  ...         ...      ...
 2803         1          71947  ...        Immm         
 2804         3         100268  ...       F-43m      300
 2805    [2, 2]         230266  ...       Fm-3m         
 2806         1         253696  ...       P2_13         
 2807         1         243641  ...        R-3c         
 
 [1464 rows x 9 columns],      bandgap value (eV)
 0                  1.05
 1                  5.2 
 2                  1.5 
 3                 2.29 
 4                 1.77 
 ...                 ...
 2803              1.48 
 2804               0.47
 2805               2.25
 2806               4.2 
 2807         

Load the materials project PBE data

In [ ]:
f = f.load('_test_mp_bandgap_v1.1', globus=False)
df_pbe_X, df_pbe_Y = f.load_data()

Loading by source_id
Done curling.
[{'/foundry/_test_mp_bandgap_v1.1/foundry_dataframe.json status': True}]


Load other computational data

In [ ]:
f = f.load('_test_comp_bandgap_v1.1',globus=False)
df_X, df_Y = f.load_data()

Loading by source_id
Done curling.
[{'/foundry/_test_comp_bandgap_v1.1/foundry_dataframe.json status': True}]


Obtain the Jarvis PBE data with OPTB88 (vdW)

In [ ]:
df_pbe1_X = df_X[(df_X['comp method'] == 'optb88') ]
df_pbe1_Y = df_Y[(df_X['comp method'] == 'optb88') ]

Obtain the data for mbj

In [ ]:
df_temp1_X = df_X[(df_X['comp method'] == 'MBJ') | (df_X['comp method'] == 'mbj')]
df_temp1_Y = df_Y[(df_X['comp method'] == 'MBJ') | (df_X['comp method'] == 'mbj')]
df_temp_X = df_temp1_X[df_temp1_X['bandgap type'] != 'D']
df_temp_Y = df_temp1_Y[df_temp1_X['bandgap type'] != 'D']
df_temp2_X = df_temp_X.copy()
df_temp2_Y = df_temp_Y.copy()
# remove repeated case
composition = []
sp = []
N = len(df_temp_X)
for i in range(N):
    str_temp = Structure.from_dict(df_temp_X.iloc[i]['structure'])
    composition.append(str_temp.composition.reduced_formula)
    sp.append(str_temp.get_space_group_info()[1])
df_no_repeat_X = []
df_no_repeat_Y = []
for com in list({value:"" for value in composition}):
    temp = []
    for index in range(N):
        if composition[index] == com:
            temp.append(index)
    if len(temp) == 1:
        df_no_repeat_X.append(df_temp_X.iloc[temp[0]])
        df_no_repeat_Y.append(df_temp_Y.iloc[temp[0]])
    else:       
        spacegroup = []
        for index in temp:
            spacegroup.append(sp[index])
        for sp1 in list({value:"" for value in spacegroup}):
            temp1 = []
            for index in temp:
                if  sp[index] == sp1:
                    temp1.append(index)
            if len(temp1) == 1:
                df_no_repeat_X.append(df_temp_X.iloc[temp1[0]])
                df_no_repeat_Y.append(df_temp_Y.iloc[temp1[0]])
            else:
                values = []
                for index in temp1:
                    values.append(float(df_temp_Y.iloc[index]['bandgap value (eV)']))
                mean = np.mean(values)
                std = np.std(values)
                if abs(mean) < 1.e-10 or abs(std) < 1.e-10:
                    df_no_repeat_Y.append(df_temp_Y.iloc[temp1[0]])
                    df_no_repeat_X.append(df_temp_X.iloc[temp1[0]])
                elif std/mean < 0.1:
                    df_temp2_Y.iloc[temp1[0],df_temp2_Y.columns.get_loc('bandgap value (eV)')] = mean
                    df_no_repeat_Y.append(df_temp2_Y.iloc[temp1[0]])
                    df_no_repeat_X.append(df_temp2_X.iloc[temp1[0]])
df_mbj_X = pd.DataFrame(df_no_repeat_X)
df_mbj_Y = pd.DataFrame(df_no_repeat_Y)

/usr/local/lib/python3.7/dist-packages/pymatgen/core/periodic_table.py:212: UserWarning: No electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  "avoid errors caused by the code expecting a float." % self.symbol
/usr/local/lib/python3.7/dist-packages/pymatgen/core/periodic_table.py:212: UserWarning: No electronegativity for Ne. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  "avoid errors caused by the code expecting a float." % self.symbol
/usr/local/lib/python3.7/dist-packages/pymatgen/core/periodic_table.py:212: UserWarning: No electronegativity for Ar. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  "avoid errors caused by the code expecting a float." % self.symbol


Obtain the data for GLLB-SC

In [ ]:
df_temp1_X = df_X[(df_X['comp method'] == 'GLLB_SC') | (df_X['comp method'] == 'GLLB-SC')]
df_temp1_Y = df_Y[(df_X['comp method'] == 'GLLB_SC') | (df_X['comp method'] == 'GLLB-SC')]
df_temp_X = df_temp1_X[df_temp1_X['bandgap type'] != 'D']
df_temp_Y = df_temp1_Y[df_temp1_X['bandgap type'] != 'D']
df_temp2_X = df_temp_X.copy()
df_temp2_Y = df_temp_Y.copy()
# remove repeated case
composition = []
sp = []
N = len(df_temp_X)
for i in range(N):
    str_temp = Structure.from_dict(df_temp_X.iloc[i]['structure'])
    composition.append(str_temp.composition.reduced_formula)
    sp.append(str_temp.get_space_group_info()[1])
df_no_repeat_X = []
df_no_repeat_Y = []
for com in list({value:"" for value in composition}):
    temp = []
    for index in range(N):
        if composition[index] == com:
            temp.append(index)
    if len(temp) == 1:
        df_no_repeat_X.append(df_temp_X.iloc[temp[0]])
        df_no_repeat_Y.append(df_temp_Y.iloc[temp[0]])
    else:       
        spacegroup = []
        for index in temp:
            spacegroup.append(sp[index])
        for sp1 in list({value:"" for value in spacegroup}):
            temp1 = []
            for index in temp:
                if  sp[index] == sp1:
                    temp1.append(index)
            if len(temp1) == 1:
                df_no_repeat_X.append(df_temp_X.iloc[temp1[0]])
                df_no_repeat_Y.append(df_temp_Y.iloc[temp1[0]])
            else:
                values = []
                for index in temp1:
                    values.append(float(df_temp_Y.iloc[index]['bandgap value (eV)']))
                mean = np.mean(values)
                std = np.std(values)
                if abs(mean) < 1.e-10 or abs(std) < 1.e-10:
                    df_no_repeat_Y.append(df_temp_Y.iloc[temp1[0]])
                    df_no_repeat_X.append(df_temp_X.iloc[temp1[0]])
                elif std/mean < 0.1:
                    df_temp2_Y.iloc[temp1[0],df_temp2_Y.columns.get_loc('bandgap value (eV)')] = mean
                    df_no_repeat_Y.append(df_temp2_Y.iloc[temp1[0]])
                    df_no_repeat_X.append(df_temp2_X.iloc[temp1[0]])
df_gllb_X = pd.DataFrame(df_no_repeat_X)
df_gllb_Y = pd.DataFrame(df_no_repeat_Y)

Obtain the data for HSE



In [ ]:
df_temp1_X = df_X[(df_X['comp method'] == 'HSE') | (df_X['comp method'] == 'hse')]
df_temp1_Y = df_Y[(df_X['comp method'] == 'HSE') | (df_X['comp method'] == 'hse')]
df_temp_X = df_temp1_X[df_temp1_X['bandgap type'] != 'D']
df_temp_Y = df_temp1_Y[df_temp1_X['bandgap type'] != 'D']
df_temp2_X = df_temp_X.copy()
df_temp2_Y = df_temp_Y.copy()
# remove repeated case
composition = []
sp = []
N = len(df_temp_X)
for i in range(N):
    str_temp = Structure.from_dict(df_temp_X.iloc[i]['structure'])
    composition.append(str_temp.composition.reduced_formula)
    sp.append(str_temp.get_space_group_info()[1])
df_no_repeat_X = []
df_no_repeat_Y = []
for com in list({value:"" for value in composition}):
    temp = []
    for index in range(N):
        if composition[index] == com:
            temp.append(index)
    if len(temp) == 1:
        df_no_repeat_X.append(df_temp_X.iloc[temp[0]])
        df_no_repeat_Y.append(df_temp_Y.iloc[temp[0]])
    else:       
        spacegroup = []
        for index in temp:
            spacegroup.append(sp[index])
        for sp1 in list({value:"" for value in spacegroup}):
            temp1 = []
            for index in temp:
                if  sp[index] == sp1:
                    temp1.append(index)
            if len(temp1) == 1:
                df_no_repeat_X.append(df_temp_X.iloc[temp1[0]])
                df_no_repeat_Y.append(df_temp_Y.iloc[temp1[0]])
            else:
                values = []
                for index in temp1:
                    values.append(float(df_temp_Y.iloc[index]['bandgap value (eV)']))
                mean = np.mean(values)
                std = np.std(values)
                if abs(mean) < 1.e-10 or abs(std) < 1.e-10:
                    df_no_repeat_Y.append(df_temp_Y.iloc[temp1[0]])
                    df_no_repeat_X.append(df_temp_X.iloc[temp1[0]])
                elif std/mean < 0.1:
                    df_temp2_Y.iloc[temp1[0],df_temp2_Y.columns.get_loc('bandgap value (eV)')] = mean
                    df_no_repeat_Y.append(df_temp2_Y.iloc[temp1[0]])
                    df_no_repeat_X.append(df_temp2_X.iloc[temp1[0]])
df_hse_X = pd.DataFrame(df_no_repeat_X)
df_hse_Y = pd.DataFrame(df_no_repeat_Y)

/usr/local/lib/python3.7/dist-packages/pymatgen/core/periodic_table.py:212: UserWarning: No electronegativity for He. Setting to NaN. This has no physical meaning, and is mainly done to avoid errors caused by the code expecting a float.
  "avoid errors caused by the code expecting a float." % self.symbol


Obtain the data for GW

In [ ]:
df_temp1_X = df_X[(df_X['comp method'] == 'GW') | (df_X['comp method'] == 'GWVD')]
df_temp1_Y = df_Y[(df_X['comp method'] == 'GW') | (df_X['comp method'] == 'GWVD')]
df_temp_X = df_temp1_X[df_temp1_X['bandgap type'] != 'D']
df_temp_Y = df_temp1_Y[df_temp1_X['bandgap type'] != 'D']
df_temp2_X = df_temp_X.copy()
df_temp2_Y = df_temp_Y.copy()
# remove repeated case
composition = []
sp = []
N = len(df_temp_X)
for i in range(N):
    str_temp = Structure.from_dict(df_temp_X.iloc[i]['structure'])
    composition.append(str_temp.composition.reduced_formula)
    sp.append(str_temp.get_space_group_info()[1])
df_no_repeat_X = []
df_no_repeat_Y = []
for com in list({value:"" for value in composition}):
    temp = []
    for index in range(N):
        if composition[index] == com:
            temp.append(index)
    if len(temp) == 1:
        df_no_repeat_X.append(df_temp_X.iloc[temp[0]])
        df_no_repeat_Y.append(df_temp_Y.iloc[temp[0]])
    else:       
        spacegroup = []
        for index in temp:
            spacegroup.append(sp[index])
        for sp1 in list({value:"" for value in spacegroup}):
            temp1 = []
            for index in temp:
                if  sp[index] == sp1:
                    temp1.append(index)
            if len(temp1) == 1:
                df_no_repeat_X.append(df_temp_X.iloc[temp1[0]])
                df_no_repeat_Y.append(df_temp_Y.iloc[temp1[0]])
            else:
                values = []
                for index in temp1:
                    values.append(float(df_temp_Y.iloc[index]['bandgap value (eV)']))
                mean = np.mean(values)
                std = np.std(values)
                if abs(mean) < 1.e-10 or abs(std) < 1.e-10:
                    df_no_repeat_Y.append(df_temp_Y.iloc[temp1[0]])
                    df_no_repeat_X.append(df_temp_X.iloc[temp1[0]])
                elif std/mean < 0.1:
                    df_temp2_Y.iloc[temp1[0],df_temp2_Y.columns.get_loc('bandgap value (eV)')] = mean
                    df_no_repeat_Y.append(df_temp2_Y.iloc[temp1[0]])
                    df_no_repeat_X.append(df_temp2_X.iloc[temp1[0]])
df_gw_X = pd.DataFrame(df_no_repeat_X)
df_gw_Y = pd.DataFrame(df_no_repeat_Y)

# Set and initiate the MEGNet model

In [ ]:
# install megnet
!pip install megnet
!pip install jsonlines

Define data fidelity

In [ ]:
ALL_FIDELITIES = ['pbe','pbe1','mbj', 'gllb-sc', 'hse','gw','exp']
TRAIN_FIDELITIES= ['pbe','pbe1','mbj', 'gllb-sc', 'hse', 'gw','exp']
VAL_FIDELITIES = ['mbj','gllb-sc', 'hse', 'gw','exp']
TEST_FIDELITIES= ['pbe','pbe1','mbj', 'gllb-sc', 'hse', 'gw','exp']

Import needed modules

In [ ]:
from megnet.callbacks import ReduceLRUponNan, ManualStop
from megnet.data.crystal import CrystalGraph
from megnet.data.graph import GraphBatchDistanceConvert, GaussianDistance
from megnet.models import MEGNetModel

# import jsonlines


import tensorflow as tf
tf.compat.v1.disable_eager_execution()

Initiate the megnet model

In [ ]:
##  Set the number of maximum epochs to 1500
EPOCHS = 1500

##  Random seed
SEED = 11742
crystal_graph = CrystalGraph(
        bond_converter=GaussianDistance(centers=np.linspace(0, 6, 100),
                                        width=0.5),
        cutoff=5.0)
model = MEGNetModel(nfeat_edge=100, nfeat_global=None, ngvocal=len(TRAIN_FIDELITIES),
                    global_embedding_dim=16,  nblocks=3, nvocal=95,
                    npass=2, graph_converter=crystal_graph, lr=1e-3)

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Set the structure feature(graph) and target (bandgap value).

In [ ]:
graphs = []
targets = []
indexs = []

for fidelity_id, fidelity in enumerate(ALL_FIDELITIES):
    if fidelity == 'hse':
        N = len(df_hse_X)
        df_temp_X = df_hse_X.copy()
        df_temp_Y = df_hse_Y.copy()
    elif fidelity == 'gw':
        N = len(df_gw_X)
        df_temp_X = df_gw_X.copy()
        df_temp_Y = df_gw_Y.copy()
    elif fidelity == 'gllb-sc':
        N = len(df_gllb_X)
        df_temp_X = df_gllb_X.copy()
        df_temp_Y = df_gllb_Y.copy()
    elif fidelity == 'exp':
        N = len(df_exp_X)
        df_temp_X = df_exp_X.copy()
        df_temp_Y = df_exp_Y.copy()
    elif fidelity == 'pbe':
        N = len(df_pbe_X)
        df_temp_X = df_pbe_X.copy()
        df_temp_Y = df_pbe_Y.copy()
    elif fidelity == 'mbj':
        N = len(df_mbj_X)
        df_temp_X = df_mbj_X.copy()
        df_temp_Y = df_mbj_Y.copy()
        
    for index in range(N):
        try:
            graph = crystal_graph.convert(Structure.from_dict(df_temp_X.iloc[index]['structure']))
            graph['state'] = [fidelity_id]
            graphs.append(graph)
            bd_value = float(df_temp_Y.iloc[index]['bandgap value (eV)'])
            targets.append(bd_value)
            indexs.append('%s_%s' % (str(index), fidelity))
        except:
            pass

final_graphs = {i:j for i, j in zip(indexs, graphs)}
final_targets = {i:j for i, j in zip(indexs, targets)}

Data split

In [ ]:
from sklearn.model_selection import train_test_split

##  train:val:test = 8:1:1
fidelity_list = [i.split('_')[1] for i in indexs]
train_val_ids, test_ids = train_test_split(indexs, stratify=fidelity_list,
                                           test_size=0.1, random_state=SEED)
fidelity_list = [i.split('_')[1] for i in train_val_ids]
train_ids, val_ids = train_test_split(train_val_ids, stratify=fidelity_list,
                                      test_size=0.1/0.9, random_state=SEED)

##  remove pbe from validation
val_ids = [i for i in val_ids if not i.endswith('pbe')]

print("Train, val and test data sizes are ", len(train_ids), len(val_ids), len(test_ids))

Train, val and test data sizes are  144456 10562 18058


In [ ]:
## Get the train, val and test graph-target pairs
def get_graphs_targets(ids):
    """
    Get graphs and targets list from the ids
    
    Args:
        ids (List): list of ids
    
    Returns:
        list of graphs and list of target values
    """
    ids = [i for i in ids if i in final_graphs]
    return [final_graphs[i] for i in ids], [final_targets[i] for i in ids]

train_graphs, train_targets = get_graphs_targets(train_ids)
val_graphs, val_targets = get_graphs_targets(val_ids)

Model training

In [ ]:
callbacks = [ReduceLRUponNan(patience=500), ManualStop()]
model.train_from_graphs(train_graphs, train_targets, val_graphs, val_targets,
                        epochs=EPOCHS, verbose=2, initial_epoch=0, callbacks=callbacks)

Epoch 1/1500


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/Adam/gradients/gradients/set2set_atom/GatherV2_1_grad/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("training/Adam/gradients/gradients/set2set_atom/GatherV2_1_grad/Reshape:0", shape=(None,), dtype=float32), dense_shape=Tensor("training/Adam/gradients/gradients/set2set_atom/GatherV2_1_grad/Cast:0", shape=(1,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/Adam/gradients/gradients/set2set_bond/GatherV2_1_grad/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("training/Adam/gradients/gradients/set2set_bond/GatherV2_1_gra